In [2]:
from bs4 import BeautifulSoup
import requests
import asyncio
import aiohttp
import json

In [117]:
response = requests.get("https://gharsansarnepal.com/buy-properties-in-nepal?page=5")
print(response)
body = BeautifulSoup(response.content, 'html.parser')
print(body.find_all('div', {'class': 'explore-item-title'}))

<Response [403]>
[<div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/house-in-kathmandu/722">Imadol,Gwarkhu House On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/Land-in-nepal/721">Kathmandu,Balaju Land On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/house-in-kathmandu/720">Sitapaila,Sita Homes House On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/Land-in-nepal/719">Mahankal House On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/house-in-kathmandu/718">Baniyatar House On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/Land-in-nepal/717">Rai Chowk Land On Sale</a>
</div>, <div class="explore-item-title same-height">
<a href="https://gharsansarnepal.com/house-in-kathmandu/716">Towkhel House On Sa

In [119]:
last_page = 42
buy_links = []
for page_num in range(1, last_page+1):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    }
    res = BeautifulSoup(requests.get("https://gharsansarnepal.com/buy-properties-in-nepal?page={}".format(page_num), headers=headers).content, 'html.parser')
    link = [x.find('a')['href'] for x in res.find_all('div', {'class': 'explore-item-title'})]
    buy_links.extend(link)

rent_links = []
last_page = 4
for page_num in range(1, last_page+1):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    }
    res = BeautifulSoup(requests.get("https://gharsansarnepal.com/gharsansar/nepal/rent?page={}".format(page_num)).content, 'html.parser')
    rent_links.extend([x.find('a')['href'] for x in res.find_all('div', {'class': 'explore-item-title'})])

In [120]:
links  = {'buy_links': list(set(buy_links)), 'rent_links': list(set(rent_links))}
with open('links.json', 'w') as f:
    json.dump(links, f, indent=4)

In [12]:
# get data from each page
with open('links.json', 'r') as f:
    links = json.load(f)
buy_links = links['buy_links']
rent_links = links['rent_links']

In [124]:
res = requests.get('https://gharsansarnepal.com/Land-in-nepal/462', headers=headers)

In [89]:
# get the section
with open('output.html', 'r') as f:
    soup = BeautifulSoup(str(f.read()), 'html.parser')

In [95]:
def get_data(soup):  
    price = soup.find('div', {"class": "banner-sub-title"}).find("h4").text
    location = soup.find('div', {'class': 'overview-sub-title'}).find('h5').text
    overview_details = soup.find('div', {'class': 'overview-details'})
    features = [x.text for x in overview_details.find('input', {'value': 'FEATURES'}).parent.find_next('ul').find_all('li')]
    property_description = [{x.find('input')['value'] : x.text} if x.find('input') != None else {'Other' : x.text}   for x in overview_details.find('input', {'value': 'Property details'}).parent.find_all_next('p')]
    amenities = [x.text for x in soup.find('div', {'class': 'amenities'}).find_all('p')]
    map_location = soup.find('div', {'class': 'map-field'}).find('iframe')['src']
    property_details = [x.text.replace('\n', '').split(":")[0].strip() + ": " + x.text.replace('\n', '').split(":")[1].strip() for x in soup.find("span", string="Property details").findNext("div").find("ul").find_all("li")]

    data = {
        'price': price,
        'location': location,
        'features': features,
        'property_description': property_description,
        'amenities': amenities,
        'map_location': map_location,
        'property_details': property_details
    }

    return data

In [96]:
get_data(soup)

{'price': 'Rs Npr 60000000/-',
 'location': ' location: Kathmandu, Baniyatar',
 'features': ['Parqueting floor and Modular kitchen with beautiful Dining space.',
  'Facing towards East/South direction.',
  '2Km from Samakhusi chowk.',
  'Inside house is full-furnished condition',
  'Parking space',
  'bunalow system',
  'Drainage and 24 hours per day water supply',
  'water solar,water filter and reserve tank is also available'],
 'property_description': [{'Ground Floor': '\xa0Open living space connected, Kitchen with Modular setup and Kitchen has extra store room plus beautiful Dining space, one Common Bedroom with outdoor space ,one Temple space and Common Bathroom.'},
  {'First Floor': '\xa0One Beautiful Master Bedroom with Balcony space and one simialr Master Bedroom,one Common Bedroom,one Common Bathroom.'},
  {'Second Floor': '\xa0One Master Bedroom one common bedroom one common bathroom space and\xa0family space and also patio space is provided in second floor.'},
  {'Top Floor'